In [1]:
import os

In [2]:
%pwd

'c:\\Users\\SRIRAM SRIPADA\\Downloads\\Credit_Churn_Prediction_with_MLFlow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\SRIRAM SRIPADA\\Downloads\\Credit_Churn_Prediction_with_MLFlow'

In [5]:
import pandas as pd
data = pd.read_csv("artifacts/data_ingestion/dataset.csv")
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,-0.326221,1,0.293517,-1.041760,-1.225848,-0.911583,0.646092,0.970243,0.021886,1,0,0
1,-0.440036,1,0.198164,-1.387538,0.117350,-0.911583,-1.547768,0.970243,0.216534,0,0,1
2,-1.536794,1,0.293517,1.032908,1.333053,2.527057,0.646092,-1.030670,0.240687,1,0,0
3,0.501521,1,0.007457,-1.387538,-1.225848,0.807737,-1.547768,-1.030670,-0.108918,0,0,0
4,2.063884,1,0.388871,-1.041760,0.785728,-0.911583,0.646092,0.970243,-0.365276,0,0,1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  float64
 1   Gender             10000 non-null  int64  
 2   Age                10000 non-null  float64
 3   Tenure             10000 non-null  float64
 4   Balance            10000 non-null  float64
 5   NumOfProducts      10000 non-null  float64
 6   HasCrCard          10000 non-null  float64
 7   IsActiveMember     10000 non-null  float64
 8   EstimatedSalary    10000 non-null  float64
 9   Exited             10000 non-null  int64  
 10  Geography_Germany  10000 non-null  int64  
 11  Geography_Spain    10000 non-null  int64  
dtypes: float64(8), int64(4)
memory usage: 937.6 KB


In [7]:
data.isnull().sum()

CreditScore          0
Gender               0
Age                  0
Tenure               0
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
Geography_Germany    0
Geography_Spain      0
dtype: int64

In [8]:
data.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Geography_Germany', 'Geography_Spain'],
      dtype='object')

## Update the entity

In [9]:
from dataclasses import dataclass
from pathlib import Path


#Config Entity 
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

## Update the configuration manager in src config

In [10]:
from src.ChurnPrediction.constants import *
from src.ChurnPrediction.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

## Update the components

In [12]:
import os
from src.ChurnPrediction import logger

In [13]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [15]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-12-03 00:06:41,703: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-03 00:06:41,706: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-03 00:06:41,735: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-03 00:06:41,737: INFO: common: created directory at: artifacts]
[2023-12-03 00:06:41,739: INFO: common: created directory at: artifacts/data_validation]
